# 4-Step Transportation Demand Model

This notebook demonstrates the custom 4-step model implementation using Brooklyn trip generators.

## Steps
1. **Trip Generation** - Calculate productions/attractions by zone
2. **Trip Distribution** - Gravity model to distribute trips
3. **Mode Choice** - Split trips by travel mode
4. **Traffic Assignment** - Assign vehicle trips to network (optional)

In [1]:
# Setup and imports
import sys
sys.path.insert(0, 'src')

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Import our custom 4-step model
from fourstep import (
    run_4step_model,
    FrictionFunction,
    TripRates,
    ModeChoiceModel
)
from fourstep.pipeline import FourStepModelConfig

import warnings
warnings.filterwarnings('ignore')

## Quick Start: Run with Defaults

The simplest way to run the model with default parameters:

In [2]:
# Run 4-step model with defaults
result = run_4step_model(
    trip_generators_file='data/trip_generators_brooklyn.csv',
    zones_file='data/zone_brooklyn_census_blocks.csv',
)

4-STEP TRANSPORTATION DEMAND MODEL

Loading input data...
  Loaded 345,265 trip generators
  Auto-detected zones CRS: EPSG:2263
  Loaded 9,855 zones

STEP 1: TRIP GENERATION
  Converting generators CRS from EPSG:4326 to EPSG:2263
  Assigning 345,265 generators to 9,855 zones...
           Assigning to nearest zone centroid...
  Assigned generators to 8,709 zones
  Calculating daily trips for purpose: all

  Trip Generation by Category:
    Local Retail                                4,614,751 ( 39.5%)
    Fast Food Restaurant without Drive Throu    1,238,531 ( 10.6%)
    Daycare (Children)                          1,173,995 ( 10.0%)
    Medical Office                                941,831 (  8.1%)
    Sit Down/High Turnover Restaurant             839,651 (  7.2%)
    Public School (Students)                      602,092 (  5.1%)
    Residential (3 or more floors)                517,396 (  4.4%)
    Destination Retail                            435,978 (  3.7%)
    Office (multi-tenant

In [3]:
# View results summary
print(result.summary())

4-STEP MODEL RESULTS SUMMARY

STEP 1: TRIP GENERATION
----------------------------------------
  Zones: 8709
  Total productions: 11,695,074
  Total attractions: 11,695,074

STEP 2: TRIP DISTRIBUTION
----------------------------------------
  Total trips: 11,695,074
  Average trip length: 3.26 miles
  Intrazonal trips: 0.4%
  Friction function: gamma

STEP 3: MODE CHOICE
----------------------------------------
  auto_driver: 7,661,884 (65.5%)
  auto_passenger: 1,893,613 (16.2%)
  transit: 1,405,729 (12.0%)
  walk: 349,430 (3.0%)
  bike: 384,418 (3.3%)
  Total vehicle trips: 7,661,884



## Customized Model Run

Configure all aspects of the model:

In [ ]:
# Create custom configuration
config = FourStepModelConfig(
    # Trip generation settings
    trip_purpose='all',           # 'all', 'HBW', 'HBO', 'NHB'
    time_period='daily',          # 'daily', 'AM', 'PM', 'midday'
    balance_method='attraction',  # 'attraction', 'production', 'average'
    
    # Trip distribution settings
    friction_function=FrictionFunction.GAMMA,  # EXPONENTIAL, POWER, GAMMA, TANNER
    friction_params={'alpha': -0.5, 'beta': 0.12},
    constraint_type='doubly',     # 'singly_production', 'singly_attraction', 'doubly'
    intrazonal_method='half_nearest',
    
    # Mode choice settings
    run_mode_choice=True,
    
    # Traffic assignment (requires network files)
    run_assignment=False,
    
    # Output
    output_dir='output/model_run',
    verbose=True
)

# Run model with custom config
result = run_4step_model(
    trip_generators_file='data/trip_generators_brooklyn.csv',
    zones_file='data/zone_brooklyn_census_blocks.csv',
    config=config
)

## Step-by-Step Execution

For more control, run each step individually:

In [ ]:
# Import individual modules
from fourstep.trip_generation import (
    assign_generators_to_zones,
    calculate_zone_trips,
    balance_trip_ends,
    create_zone_arrays,
    TripRates
)
from fourstep.trip_distribution import (
    calculate_impedance_matrix,
    gravity_model,
    calibrate_friction_parameter,
    FrictionFunction,
    calculate_trip_length_distribution
)
from fourstep.mode_choice import run_mode_choice, ModeChoiceModel

In [ ]:
# Load data
generators = pd.read_csv('data/trip_generators_brooklyn.csv')
zones = pd.read_csv('data/zone_brooklyn_census_blocks.csv')

# Convert to GeoDataFrames
from shapely.geometry import Point
from shapely import wkt

geometry = [Point(xy) for xy in zip(generators['lon'], generators['lat'])]
generators_gdf = gpd.GeoDataFrame(generators, geometry=geometry, crs='EPSG:4326')

zones['geometry'] = zones['geometry'].apply(wkt.loads)
zones_gdf = gpd.GeoDataFrame(zones, geometry='geometry', crs='EPSG:4326')

print(f"Trip generators: {len(generators_gdf):,}")
print(f"Zones: {len(zones_gdf):,}")

### Step 1: Trip Generation

In [ ]:
# Assign generators to zones
generators_with_zones = assign_generators_to_zones(
    generators_gdf, 
    zones_gdf,
    zone_id_col='zone_id',
    verbose=True
)

In [ ]:
# Calculate zone productions and attractions
zone_trips = calculate_zone_trips(
    generators_with_zones,
    zone_id_col='zone_id',
    trip_gen_category_col='trip_gen_category',
    trip_gen_value_col='trip_gen_value',
    trip_purpose='all',
    time_period='daily',
    verbose=True
)

zone_trips.head(10)

In [ ]:
# Balance trip ends
zone_trips_balanced = balance_trip_ends(
    zone_trips, 
    method='attraction',
    verbose=True
)

# Convert to arrays for gravity model
zone_ids, productions, attractions = create_zone_arrays(zone_trips_balanced, 'zone_id')
print(f"\nZones with trips: {len(zone_ids)}")

### Step 2: Trip Distribution

In [ ]:
# Filter zones to those with trips
zones_with_trips = zones_gdf[zones_gdf['zone_id'].isin(zone_ids)]

# Calculate impedance (distance) matrix
zone_ids_imp, impedance = calculate_impedance_matrix(
    zones_with_trips,
    zone_id_col='zone_id',
    intrazonal_method='half_nearest',
    verbose=True
)

In [ ]:
# Reorder arrays to match impedance matrix
zone_order = {zid: i for i, zid in enumerate(zone_ids_imp)}
reorder_idx = [zone_order[zid] for zid in zone_ids if zid in zone_order]

# Filter to matching zones
valid_mask = np.isin(zone_ids, zone_ids_imp)
zone_ids = zone_ids[valid_mask]
productions = productions[valid_mask]
attractions = attractions[valid_mask]

# Reorder impedance matrix
reorder_idx = [zone_order[zid] for zid in zone_ids]
impedance = impedance[np.ix_(reorder_idx, reorder_idx)]

print(f"Final zones: {len(zone_ids)}")
print(f"Impedance matrix: {impedance.shape}")

In [ ]:
# Run gravity model with different friction functions

# Option 1: Exponential friction
trips_exp, diag_exp = gravity_model(
    productions, attractions, impedance,
    friction_function=FrictionFunction.EXPONENTIAL,
    friction_params={'beta': 0.15},
    constraint_type='doubly',
    verbose=True
)

In [ ]:
# Option 2: Gamma friction (more flexible)
trips_gamma, diag_gamma = gravity_model(
    productions, attractions, impedance,
    friction_function=FrictionFunction.GAMMA,
    friction_params={'alpha': -0.5, 'beta': 0.1},
    constraint_type='doubly',
    verbose=True
)

In [ ]:
# Option 3: Calibrate to target average trip length
target_trip_length = 3.5  # miles

calibrated_beta, trips_calibrated, calib_diag = calibrate_friction_parameter(
    productions, attractions, impedance,
    target_avg_trip_length=target_trip_length,
    friction_function=FrictionFunction.EXPONENTIAL,
    parameter_name='beta',
    initial_value=0.1,
    verbose=True
)

print(f"\nCalibrated beta: {calibrated_beta:.6f}")

In [ ]:
# Compare trip length distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (trips, name) in zip(axes, [
    (trips_exp, 'Exponential'),
    (trips_gamma, 'Gamma'),
    (trips_calibrated, f'Calibrated (target={target_trip_length}mi)')
]):
    bins, counts, pcts = calculate_trip_length_distribution(trips, impedance, n_bins=20)
    ax.bar(bins[:-1], pcts, width=np.diff(bins), align='edge', alpha=0.7)
    ax.set_xlabel('Trip Length (miles)')
    ax.set_ylabel('Percentage of Trips')
    ax.set_title(f'{name}\nAvg: {(trips * impedance).sum() / trips.sum():.2f} mi')

plt.tight_layout()
plt.show()

### Step 3: Mode Choice

In [ ]:
# Run mode choice on the calibrated trip matrix
mode_trips, vehicle_trips = run_mode_choice(
    trips_calibrated,
    impedance,
    model=None,  # Use defaults
    verbose=True
)

In [ ]:
# Visualize mode shares
total_trips = sum(m.sum() for m in mode_trips.values())
mode_shares = {mode: trips.sum() / total_trips * 100 for mode, trips in mode_trips.items()}

fig, ax = plt.subplots(figsize=(10, 6))
modes = list(mode_shares.keys())
shares = list(mode_shares.values())

bars = ax.barh(modes, shares, color=['#2ecc71', '#3498db', '#9b59b6', '#e74c3c', '#f39c12'])
ax.set_xlabel('Mode Share (%)')
ax.set_title('Mode Split Results')

for bar, share in zip(bars, shares):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, 
            f'{share:.1f}%', va='center')

plt.tight_layout()
plt.show()

## Gravity Model Options Summary

The gravity model supports these improvements via function parameters:

| Parameter | Options | Description |
|-----------|---------|-------------|
| `friction_function` | EXPONENTIAL, POWER, GAMMA, TANNER | Decay function type |
| `friction_params` | dict | Parameters for chosen function |
| `constraint_type` | singly_production, singly_attraction, doubly | Balancing method |
| `k_factors` | np.ndarray | Zone-pair adjustment matrix |
| `accessibility_weight` | float | Hansen accessibility weighting |

In [ ]:
# Example: Using K-factors for barrier adjustments
# Create K-factor matrix (1.0 = no adjustment)
n_zones = len(zone_ids)
k_factors = np.ones((n_zones, n_zones))

# Example: Reduce trips across a hypothetical barrier by 30%
# k_factors[barrier_origins, barrier_dests] = 0.7

trips_with_k, _ = gravity_model(
    productions, attractions, impedance,
    friction_function=FrictionFunction.GAMMA,
    friction_params={'alpha': -0.5, 'beta': 0.1},
    constraint_type='doubly',
    k_factors=k_factors,  # Apply K-factors
    verbose=True
)

In [ ]:
# Example: Accessibility-weighted attractions
trips_acc, _ = gravity_model(
    productions, attractions, impedance,
    friction_function=FrictionFunction.GAMMA,
    friction_params={'alpha': -0.5, 'beta': 0.1},
    constraint_type='doubly',
    accessibility_weight=0.5,  # Boost high-accessibility zones
    verbose=True
)

## Save Results

In [ ]:
# Save OD matrix to CSV
import os
os.makedirs('output', exist_ok=True)

# Create OD dataframe
od_records = []
for i in range(len(zone_ids)):
    for j in range(len(zone_ids)):
        if trips_calibrated[i, j] > 0:
            od_records.append({
                'origin_zone': zone_ids[i],
                'dest_zone': zone_ids[j],
                'trips': trips_calibrated[i, j],
                'distance_miles': impedance[i, j]
            })

od_df = pd.DataFrame(od_records)
od_df.to_csv('output/od_matrix.csv', index=False)
print(f"Saved {len(od_df):,} OD pairs to output/od_matrix.csv")